In [115]:
#reddit api

import praw

In [116]:
#download method
import requests

#other
import pprint
import re
from datetime import date
import time

# File management
import os, os.path
import json
from PIL import Image

In [117]:
def downloadImage(imageUrl, localFileName):
    response = requests.get(imageUrl)
#     print(imageUrl)
#     if response.status_code == 200:
#         print('Downloading %s...' % (localFileName))
    with open(localFileName, 'wb') as fo:
        for chunk in response.iter_content(4096):
            fo.write(chunk)

In [118]:
#https://stackoverflow.com/questions/8032642/
# how-to-obtain-image-size-using-standard-python-class-without-using-external-lib/20380514#20380514
import struct
import imghdr

def test_jpeg(h, f):
    # SOI APP2 + ICC_PROFILE
    if h[0:4] == '\xff\xd8\xff\xe2' and h[6:17] == b'ICC_PROFILE':
#         print "A"
        return 'jpeg'
    # SOI APP14 + Adobe
    if h[0:4] == '\xff\xd8\xff\xee' and h[6:11] == b'Adobe':
        return 'jpeg'
    # SOI DQT
    if h[0:4] == '\xff\xd8\xff\xdb':
        return 'jpeg'
imghdr.tests.append(test_jpeg)

def get_image_size(fname):
    '''Determine the image type of fhandle and return its size.
    from draco'''
    with open(fname, 'rb') as fhandle:
        head = fhandle.read(24)
        if len(head) != 24:
            return
        what = imghdr.what(None, head)
        if what == 'png':
            check = struct.unpack('>i', head[4:8])[0]
            if check != 0x0d0a1a0a:
                return
            width, height = struct.unpack('>ii', head[16:24])
        elif what == 'gif':
            width, height = struct.unpack('<HH', head[6:10])
        elif what == 'jpeg':
            try:
                fhandle.seek(0) # Read 0xff next
                size = 2
                ftype = 0
                while not 0xc0 <= ftype <= 0xcf or ftype in (0xc4, 0xc8, 0xcc):
                    fhandle.seek(size, 1)
                    byte = fhandle.read(1)
                    while ord(byte) == 0xff:
                        byte = fhandle.read(1)
                    ftype = ord(byte)
                    size = struct.unpack('>H', fhandle.read(2))[0] - 2
                # We are at a SOFn block
                fhandle.seek(1, 1)  # Skip `precision' byte.
                height, width = struct.unpack('>HH', fhandle.read(4))
            except Exception: #IGNORE:W0703
                return
        else:
            return
        return width, height

In [119]:
def get_caption(comment):
    return re.split('[.!?\n]',comment)[0]

In [120]:
reddit = praw.Reddit(client_id='NLIznGitXU82bA',
                     client_secret='in6SUevIRf3dLYHqu2YFH8xYAq8',
                     password='willmarkandmatt',
                     user_agent='script2',
                     username='231ntest1')

In [121]:
print(reddit.user.me())

231ntest1


In [122]:
print(reddit.read_only)  # Output: False

False


In [123]:
for submission in reddit.subreddit('pics').top("all",limit=10):
    print(submission.title)

Private Internet Access, a VPN provider, takes out a full page ad in The New York Time calling out 50 senators.
Thanks, Obama.
This is Shelia Fredrick, a flight attendant. She noticed a terrified girl accompanied by an older man. She left a note in the bathroom on which the victim wrote that she needed help. The police was alerted & the girl was saved from a human trafficker. We should honor our heroes.
And here it is: Likely the last sketch I'll ever post on Reddit. It's been a wild ride, guys and girls. Thanks for everything.
This image is now illegal in Russia.
Should have been Bernie
I've been photoshopping my kid into marginally dangerous situations. Nothing unbelievable, but enough to make people think "Wait, did he..?"
I wander the streets of Toronto at night, looking for cinematic moments. This is what I found...
Don't forget about this
First and last day of 1st grade- she kicked cancers ass!


In [124]:
subName = "aww"
substats ={
    "title" : "rarepup",
    "comments" : ["wow", "such rare", "so pup"]
}
json.dumps(substats)
a = "hello? yes. it's me!"
re.split("[?.!]", a)



['hello', ' yes', " it's me", '']

In [127]:
# Extract comment + image pairs from subreddit

info = {
    "year" : 2017,
    "version" : 0,
    "description" : "CS231N Toy Dataset",
    "contributor" : "Mathieu Rolfo, Mark Nishimura, William Clary",
    "url" : "",
    "date_created" : str(date.today()),
}

unlicensed_license = {
    "id" : 0,
    "name" : "Unlicensed",
    "url" : "",
}

licenses = [unlicensed_license]
images = []
annotations = []
image_to_annotations = {}
nposts = 10000
commentsperpost = 5
subreddits = ["rarepuppers", "aww", "eyebleach"]
now = time.time()
rawFolder = "raw"

captionID = 0 # Initialize to 0
for subName in subreddits:
    print("Extracting data from r/" + subName + "...")
    subImages = []
    subCaptions = []
    for submission in reddit.subreddit(subName).top("all", limit=nposts):
        #pprint.pprint(vars(submission))
    #     print(submission.comments)
        try:
            submission.comments.replace_more(limit=0)
        except:
            continue
    #     for top_level_comment in submission.comments[:5]:
    #         # Extract first sentence of each top comment.
    #         print(re.split('[.!?]',top_level_comment.body)[0]) 
    
        # Filter out gifs, albums
        if not submission.url.endswith(('jpg', 'jpeg')):
            continue
        # Download image and attempt to extract width and height:
        filename = submission.id + ".jpg"
        try:
            downloadImage(submission.url, os.path.join(rawFolder, filename))
        except:
            # Just skip this image:
            continue
        size = get_image_size(os.path.join(rawFolder, filename))
        if size is None:
            # Manually process image by opening/reading it:
            im = Image.open(os.path.join(rawFolder, filename))
            size = im.size
        width, height = size

        # Extract imageID from reddit's base 36
        imageID = int(submission.id, 36)

        # Generate Image and Annotation json
        captions = [
            {
                "id" : captionID + k,
                "image_id" : imageID,
                "caption" : get_caption(top_level_comment.body),
            }
            for k, top_level_comment in enumerate(submission.comments[:commentsperpost])
        ]
        captionID += commentsperpost
        # Include submission title
        captions += [
            {
                "id" : captionID,
                "image_id" : imageID,
                "caption" : submission.title
            }
        ]
        captionID += 1
        
        # Add captions and image to lists:
        subCaptions += captions
        image = {
            "id" : imageID,
            "width" : width,
            "height" : height,
            "file_name" : filename,
            "license" : 0,
            "flickr_url" : "",
            "coco_url" : "",
            "date_captured" : str(date.today()),
        }
        subImages += [image]
        # Keep track of which images get assigned to which captions:
        image_to_annotations[imageID] = captions
    print("\tRetrieved "+ str(len(subImages)) + " images from r/" + subName + ".")
    print("\tRetrieved "+ str(len(subCaptions)) + " captions from r/" + subName + ".")
    images+= subImages
    annotations += subCaptions
    
# out = {
#     "info" : info,
#     "images" : images,
#     "annotations" : annotations,
#     "licenses" : licenses
# }
print("Retrieved "+ str(len(images)) + " total images.")
print("Retrieved "+ str(len(annotations)) + " total captions")
print("Done. (" + str(time.time() - now) + " s)")

Extracting data from r/rarepuppers...
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
	Retrieved 491 images from r/rarepuppers.
	Retrieved 2941 captions from r/rarepuppers.
Extracting data from r/aww...
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
	Retrieved 305 images from r/aww.
	Retrieved 1830 captions from r/aww.
Extracting data from r/eyebleach...
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
	Retrieved 218 images from r/eyebleach.
	Retrieved 1307 captions from r/eyebleach.
Retrieved 1014 total images.
Retrieved 6078 total captions
Done. (5598.24039412 s)


In [128]:
import random
num_images = len(images)
num_val = num_images/10 #9:1 testing/training ratio
# Sample the indices:
val_ind = random.sample(range(num_images), num_val)
train_ind = [i for i in range(num_images) if i not in val_ind]
# Split data into training and evaluation sets:
val_images = [images[i] for i in val_ind] 
val_annotations = []
for image in val_images:
    val_annotations += image_to_annotations[image["id"]]
train_images = [images[i] for i in train_ind ]
train_annotations = []
for image in train_images:
    train_annotations += image_to_annotations[image["id"]]

# for i in random.sample(range(num_images), num_val):
#     image_id = images[i]["id"]
#     val_images.append(images[i])
#     train_images.remove(images[i])
#     # Move images to the correct directory
#     filename = images[i]["file_name"]

print("Created training and validation sets.")
print("Training set: ")
print("\t"+str(len(train_images))+" images, " + str(len(train_annotations))+ " captions.")
print("Validation set: ")
print("\t"+str(len(val_images))+" images, " + str(len(val_annotations))+ " captions.")     

train_json = {
    "info" : info,
    "images" : train_images,
    "annotations" : train_annotations,
    "licenses" : licenses
}

val_json = {
    "info" : info,
    "images" : val_images,
    "annotations" : val_annotations,
    "licenses" : licenses
}

# print("train images:")
# for image in train_images:
#     print image["id"]
# print("val images:")
# for image in val_images:
#     print image["id"]
    
# print("train captions:")
# for caption in train_annotations:
#     print caption["image_id"]
# print("val captions:")
# for caption in val_annotations:
#     print caption["image_id"]

Created training and validation sets.
Training set: 
	913 images, 5472 captions.
Validation set: 
	101 images, 606 captions.


In [129]:
# Separate images into folders and make json files
trainFolder = "reddit_train"
valFolder = "reddit_val"
for image in train_images:
    filename = image["file_name"]
    os.rename(os.path.join(rawFolder, filename), os.path.join(trainFolder, filename)) # Move to new location
for image in val_images:
    filename = image["file_name"]
    os.rename(os.path.join(rawFolder, filename), os.path.join(valFolder, filename)) # Move to new location   
with open('captions_train.json', 'w') as trainFile:
    json.dump(train_json, trainFile)
with open('captions_val.json', 'w') as valFile:
    json.dump(val_json, valFile)

In [142]:
from PIL import Image

rescaledTrainFolder = "reddit_train_rescaled"
rescaledValFolder = "reddit_val_rescaled"
rescaled_train_images = []
rescaled_train_annotations = []
rescaled_val_images = []
rescaled_val_annotations = []
# Rescale images that are too large:
maxsize = (640, 640)
for image in train_images:
    filename = image["file_name"]
    try:
        im = Image.open(os.path.join(trainFolder, filename))
        im.thumbnail(maxsize, Image.ANTIALIAS)
        im.save(os.path.join(rescaledTrainFolder, filename), "JPEG")
        image["width"], image["height"] = im.size
        rescaled_train_images.append(image)
        rescaled_train_annotations += image_to_annotations[image["id"]]
    except IOError:
        print("cannot create thumbnail for '%s'" % filename)
        
        
for image in val_images:
    filename = image["file_name"]
    try:
        im = Image.open(os.path.join(valFolder, filename))
        im.thumbnail(maxsize, Image.ANTIALIAS)
        im.save(os.path.join(rescaledValFolder, filename), "JPEG")
        image["width"], image["height"] = im.size
        rescaled_val_images.append(image)
        rescaled_val_annotations += image_to_annotations[image["id"]]
    except IOError:
        print("cannot create thumbnail for '%s'" % filename)

rescaled_train_json = {
    "info" : info,
    "images" : rescaled_train_images,
    "annotations" : rescaled_train_annotations,
    "licenses" : licenses
}

rescaled_val_json = {
    "info" : info,
    "images" : rescaled_val_images,
    "annotations" : rescaled_val_annotations,
    "licenses" : licenses
}

with open('rescaled_captions_train.json', 'w') as trainFile:
    json.dump(rescaled_train_json, trainFile)
with open('rescaled_captions_val.json', 'w') as valFile:
    json.dump(rescaled_val_json, valFile)



cannot create thumbnail for '5sdon0.jpg'
cannot create thumbnail for '51kvfb.jpg'
cannot create thumbnail for '5tl4rh.jpg'
cannot create thumbnail for '4wklx5.jpg'
cannot create thumbnail for '4t90ny.jpg'
cannot create thumbnail for '5l9d29.jpg'
cannot create thumbnail for '4s6izy.jpg'
cannot create thumbnail for '4yg02e.jpg'
cannot create thumbnail for '3k3mao.jpg'
cannot create thumbnail for '5exgh0.jpg'
cannot create thumbnail for '4uo7y4.jpg'
cannot create thumbnail for '4xdmjh.jpg'
cannot create thumbnail for '2jb8kx.jpg'
cannot create thumbnail for '4o5ki1.jpg'
cannot create thumbnail for '4msbwt.jpg'
cannot create thumbnail for '4eg1ix.jpg'
cannot create thumbnail for '4l5mdx.jpg'
cannot create thumbnail for '4np4dy.jpg'
cannot create thumbnail for '4k2lu3.jpg'
cannot create thumbnail for '60jxm8.jpg'
cannot create thumbnail for '4mne06.jpg'
cannot create thumbnail for '3hg5ap.jpg'
cannot create thumbnail for '3rw2qq.jpg'
cannot create thumbnail for '4yil2w.jpg'
cannot create th

In [134]:
# Testing only: Make small data set

from shutil import copyfile
smallTrainFolder = "smalldata_train"
smallValFolder = 'smalldata_val'

small_train_annotations = []
small_val_annotations = []

for image in train_images[:10]:
    filename = image["file_name"]
    copyfile(os.path.join(trainFolder, filename), os.path.join(smallTrainFolder,filename))
    small_train_annotations += image_to_annotations[image["id"]]
for image in val_images[:2]:
    filename = image["file_name"]
    copyfile(os.path.join(valFolder, filename), os.path.join(smallValFolder,filename))
    small_val_annotations += image_to_annotations[image["id"]]

smalltrain_json = {
    "info" : info,
    "images" : train_images[:10],
    "annotations" : small_train_annotations,
    "licenses" : licenses
}

smallval_json = {
    "info" : info,
    "images" : val_images[:2],
    "annotations" : small_val_annotations,
    "licenses" : licenses
}
with open('smallcaptions_train.json', 'w') as trainFile:
    json.dump(smalltrain_json, trainFile)
with open('smallcaptions_val.json', 'w') as valFile:
    json.dump(smallval_json, valFile)